In [1]:
!pip install mmdet==2.24.1 mmpose==0.25.1 opencv-python-headless==4.8.0.74 openmim==0.1.5 timm==0.5.4 torch==1.11.0 torchvision==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.0/436.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 8.2 MB/s eta 0:00:00
Reason for being yanked: deprecated, use 4.8.0.76
  Created wheel for openmim: filename=openmim-0.1.5-py2.py3-none-any.whl size=42472 sha256=fcea02350a1990516aeaa06b423ffa432d7b1898b84b9d9cded

In [2]:
!mim install mmcv-full==1.5.0

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
installing mmcv-full from wheel.
Looking in links: https://download.openmmlab.com/mmcv/dist/cpu/torch1.11.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 1.9 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/ViTAE-Transformer/ViTPose.git

Cloning into 'ViTPose'...
remote: Enumerating objects: 1859, done.
remote: Counting objects: 100% (1041/1041), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 1859 (delta 794), reused 725 (delta 725), pack-reused 818
Receiving objects: 100% (1859/1859), 10.75 MiB | 30.56 MiB/s, done.
Resolving deltas: 100% (958/958), done.


In [4]:
# !git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 38019, done.
remote: Total 38019 (delta 0), reused 0 (delta 0), pack-reused 38019
Receiving objects: 100% (38019/38019), 63.18 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (26232/26232), done.


In [4]:
!pip install yapf==0.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: yapf
    Found existing installation: yapf 0.40.2
    Uninstalling yapf-0.40.2:
      Successfully uninstalled yapf-0.40.2


In [8]:
#!mim search mmdet --model "yolox"

In [5]:
!mim download mmdet --config yolox_tiny_8x8_300e_coco --dest /content

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
processing yolox_tiny_8x8_300e_coco...
  [####################################]  100%          
Successfully downloaded yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth to /content
Successfully dumped yolox_tiny_8x8_300e_coco.py to /content


In [7]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from huggingface_hub import hf_hub_download
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model, process_mmdet_results, vis_pose_result
import urllib.request

# 物体検出モデルの初期化
class DetModel:

    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()

    def _load_model(self):
        config_path = "/content/yolox_tiny_8x8_300e_coco.py"
        model_path = "/content/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth"

        return init_detector(config_path, model_path, device=self.device)

    def detect(self, image: np.ndarray):
        image = image[:, :, ::-1]  # RGB -> BGR
        out = inference_detector(self.model, image)
        return out

    def visualize_detection_results(self, image: np.ndarray, detection_results: list, score_threshold: float = 0.3):
        person_det = [detection_results[0]] + [np.array([]).reshape(0, 5)] * 79
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = self.model.show_result(
            image, person_det, score_thr=score_threshold, bbox_color=None, text_color=(200, 200, 200), mask_color=None
        )
        return vis[:, :, ::-1]  # BGR -> RGB



det_model = DetModel()

load checkpoint from local path: /content/yolox_tiny_8x8_300e_coco_20211124_171234-b4047906.pth


In [12]:
!mim search mmpose --model "resnet"

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
--------------------------------------------------------------------------------
config id: deeppose_res50_coco_256x192
architecture                     deeppose,resnet
coco/ap                          0.526
coco/ap@0.5                      0.816
coco/ap@0.75                     0.586
coco/ar                          0.638
coco/ar@0.5                      0.887
config                           configs/body/2d_kpt_sview_rgb_img/deeppose/c...
model                            resnet
paper                            Title,URL
readme                           https://github.com/open-mmlab/mmpose/blob/ma...
training_data                    coco
weight                           https://download.openmmlab.com/mmpose/top_do...
--------------------------------------------------------------------------------
config id: deeppo

In [10]:
# 姿勢推定モデルの初期化
class PoseModel:
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model()

    def _load_model(self):
        ckpt_path = hf_hub_download("public-data/ViTPose", "models/vitpose-b.pth")
        config_path = "ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/ViTPose_base_coco_256x192.py"
        model = init_pose_model(config_path, ckpt_path, device=self.device)
        return model

    def predict_pose(self, image: np.ndarray, det_results: list, box_score_threshold: float = 0.5):
        image = image[:, :, ::-1]  # RGB -> BGR
        person_results = process_mmdet_results(det_results, 1)
        out, _ = inference_top_down_pose_model(
            self.model, image, person_results=person_results, bbox_thr=box_score_threshold, format="xyxy"
        )
        return out

    def visualize_pose_results(self, image: np.ndarray, pose_results: list, kpt_score_threshold: float = 0.3, vis_dot_radius: int = 4, vis_line_thickness: int = 1):
        image = image[:, :, ::-1]  # RGB -> BGR
        vis = vis_pose_result(
            self.model,
            image,
            pose_results,
            kpt_score_thr=kpt_score_threshold,
            radius=vis_dot_radius,
            thickness=vis_line_thickness,
        )
        return vis[:, :, ::-1]  # BGR -> RGB

pose_model = PoseModel()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vitpose-b.pth:   0%|          | 0.00/360M [00:00<?, ?B/s]

KeyError: "TopDown: 'ViT is not in the models registry'"

In [17]:
# 画像のアップロード
uploaded = files.upload()

for filename in uploaded.keys():
    image_path = filename

# 画像の読み込み
image = plt.imread(image_path)

Saving 佐藤_妖精.png to 佐藤_妖精.png


In [18]:
# 物体検出
det_results = det_model.detect(image)
det_visualization = det_model.visualize_detection_results(image, det_results)

# 検出結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(det_visualization)
plt.title("Detection Results")
plt.axis("off")
plt.show()

AssertionError: 

In [ ]:
# 姿勢推定
pose_results = pose_model.predict_pose(image, det_results)
pose_visualization = pose_model.visualize_pose_results(image, pose_results)

# 姿勢推定結果の表示
plt.figure(figsize=(10, 10))
plt.imshow(pose_visualization)
plt.title("Pose Estimation Results")
plt.axis("off")
plt.show()